## We import the necesary libraries and load the given dataset (seeds_dataset.txt)

In [36]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from scipy import stats
from sklearn import metrics

#Extraction
labels= ["Area","Perimeter","Compactness","Length_of_kernel","Width_of_kernel","Asymetry_coefficient","Length_of_kernel_groove","Class"]
data = pd.read_csv('seeds_dataset.txt', delimiter="\t", header=None, names = labels)
data.head()

,Area,Perimeter,Compactness,Length_of_kernel,Width_of_kernel,Asymetry_coefficient,Length_of_kernel_groove,Class
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


### We analyze the satate of the dataset 

In [27]:
print(data.describe())
print("\n")
print("Checking NULLs",data.isnull().sum())
#There is no nulls in the dataset
print

target = data["Class"]
X = data.drop("Class", 1)
# We Divide the target "Class" from the rest of the variables 
print("\n")
print("All data Shape -->",data.shape)
print("\n")
print("Target Shape -->",target.shape)
print("Variables Shape -->",X.shape)

             Area   Perimeter  Compactness  Length_of_kernel  Width_of_kernel  \
count  210.000000  210.000000   210.000000        210.000000       210.000000   
mean    14.847524   14.559286     0.870999          5.628533         3.258605   
std      2.909699    1.305959     0.023629          0.443063         0.377714   
min     10.590000   12.410000     0.808100          4.899000         2.630000   
25%     12.270000   13.450000     0.856900          5.262250         2.944000   
50%     14.355000   14.320000     0.873450          5.523500         3.237000   
75%     17.305000   15.715000     0.887775          5.979750         3.561750   
max     21.180000   17.250000     0.918300          6.675000         4.033000   

       Asymetry_coefficient  Length_of_kernel_groove       Class  
count            210.000000               210.000000  210.000000  
mean               3.700201                 5.408071    2.000000  
std                1.503557                 0.491480    0.818448  
mi

In [28]:
target.value_counts()
#We have 3 groups (1,2,3)
#We see that the classifications of the dataset are equally distributed (70 per each group)

3    70
2    70
1    70
Name: Class, dtype: int64

In [29]:
data.corr(method = 'pearson')
#Checking the correlation between variables we see that 
#[['Perimenter','Area']], 0.994341 , 
#[['Length_of_kernel','Area']]

,Area,Perimeter,Compactness,Length_of_kernel,Width_of_kernel,Asymetry_coefficient,Length_of_kernel_groove,Class
Area,1.000000,0.994341,0.608288,0.949985,0.970771,-0.229572,0.863693,-0.346058
Perimeter,0.994341,1.000000,0.529244,0.972422,0.944829,-0.217340,0.890784,-0.327900
Compactness,0.608288,0.529244,1.000000,0.367915,0.761635,-0.331471,0.226825,-0.531007
Length_of_kernel,0.949985,0.972422,0.367915,1.000000,0.860415,-0.171562,0.932806,-0.257269
Width_of_kernel,0.970771,0.944829,0.761635,0.860415,1.000000,-0.258037,0.749131,-0.423463
Asymetry_coefficient,-0.229572,-0.217340,-0.331471,-0.171562,-0.258037,1.000000,-0.011079,0.577273
Length_of_kernel_groove,0.863693,0.890784,0.226825,0.932806,0.749131,-0.011079,1.000000,0.024301
Class,-0.346058,-0.327900,-0.531007,-0.257269,-0.423463,0.577273,0.024301,1.000000


#### Checking the correlation between variables we see high correlation between some variables
 ['Perimenter','Area'], 0.994341 \
 ['Length_of_kernel','Area'], 0.949985 \
 ['Length_of_kernel','Perimeter'], 0.972422	 \
 ['Width_of_kernel','Area'], 0.970771	 \
 ['Width_of_kernel','Perimeter'], 0.944829   \
 ['Length_of_kernel_groove','Length_of_kernel'], 0.932806 \
 \
 Then we see the 'Area', 'Perimeter' variables have high correlation with 'Length_of_kernel' or 'Width_of_kernel' and  Length_of_kernel_groove' variable has high correlation with 'Length_of_kernel'. Then we are going to create a model not using the variables 'Perimeter', 'Area' and 'Length_of_kernel' to compare our choosen models with the dataset with all variables included.

In [30]:
light_data = data.drop(['Class','Area','Perimeter','Length_of_kernel'], 1)
#the target will not change 
light_data.head()
light_data.shape

(210, 4)

# We choose and create the models

### Naive Bayes
We are using the Naive Bayes classificator due to the DataShet is not very big and the target parameters are equally distributed.

In [43]:
#checking performance against the own train set

def modelo_NB_train(X,Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    gnb = GaussianNB()
    #train
    y_pred = gnb.fit(X_train, y_train).predict(X_train)
    print('Train__The Accuracy is: ',accuracy_score(y_train, y_pred))
    print('Train__conf_matrx:')
    print(confusion_matrix(y_train, y_pred))
#checking performance against the test

def modelo_NB_test(X,Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42) #same random state
    gnb = GaussianNB()
    #test
    y_pred = gnb.fit(X_train, y_train).predict(X_test)
    print('Test__The Accuracy is: ',accuracy_score(y_test, y_pred))
    print('Test__conf_matrx:')
    print(confusion_matrix(y_test, y_pred))
    print("\n")
    print("Classification report:")
    print(metrics.classification_report(y_test, y_pred, digits=3))

In [44]:
#First we do perform the model with all the variables:
#modelo_NB_train(X,target)
modelo_NB_test(X,target)

Test__The Accuracy is:  0.8571428571428571
Test__conf_matrx:
[[ 8  1  2]
 [ 1 13  0]
 [ 2  0 15]]


Classification report:
              precision    recall  f1-score   support

           1      0.727     0.727     0.727        11
           2      0.929     0.929     0.929        14
           3      0.882     0.882     0.882        17

    accuracy                          0.857        42
   macro avg      0.846     0.846     0.846        42
weighted avg      0.857     0.857     0.857        42



In [45]:
#Now with the dropped values
#modelo_NB_train(light_data,target)
modelo_NB_test(light_data,target)

Test__The Accuracy is:  0.8809523809523809
Test__conf_matrx:
[[10  0  1]
 [ 0 14  0]
 [ 4  0 13]]


Classification report:
              precision    recall  f1-score   support

           1      0.714     0.909     0.800        11
           2      1.000     1.000     1.000        14
           3      0.929     0.765     0.839        17

    accuracy                          0.881        42
   macro avg      0.881     0.891     0.880        42
weighted avg      0.896     0.881     0.882        42



### Naive Bayes model Conclusion:

Having the 'modelo_NB_test' function results as valid we can see how avoiding variables that are highly correlated with others we obtain abetter results.
Complete Variables Dataset --> Accuracy = 0.8571428571428571 \
Suppressed Variables Dataset --> Accuracy = 0.8809523809523809 \
\
In the Classification report we can also see how the precision improves on each class in the second case.

Also we can see in the Confusion matrix how in the first case we get more false positives when using all the variables among the different classes, where in the second case, using less variables, the number of false positives decreaseas.

